In [1]:
import pickle

with open('/kaggle/input/dictionary/dictionary.pkl', 'rb') as f:
    inverted_index = pickle.load(f)

In [2]:
type(inverted_index)

dict

In [9]:
inverted_index['goldberg']

[12,
 {0: [1],
  1: [111],
  2: [4],
  3: [11],
  4: [5, 35],
  6: [2, 27],
  89695: [169],
  274490: [44],
  383088: [142],
  396714: [191],
  467580: [43],
  467581: [55, 72]},
 15]

In [3]:
import re

def apply_preprocessing(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Replace whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    
    return text

In [32]:
query = input("Enter your query: ")

Enter your query:  like 10 love


In [33]:
query

'like 10 love'

In [34]:
# check if the query is a phrase or proximity query
if query[0] == '"' and query[-1] == '"':
    query = apply_preprocessing(query[1:-1])
    query = query.split()
    q_type = 'PHRASE'
else:
    try:
        query = apply_preprocessing(query)
        l = query.split()
        k = int(l[1]) + 1
        query = [l[0], l[2]]
        q_type = 'PROXIMITY'
    except:
        print('Try this format: string1 integer string2')

In [35]:
try:
    freqs = [(inverted_index[word][0], index, word) for index, word in enumerate(query)]
except:
    print('Word not found in dictionary')

In [36]:
freqs

[(220863, 0, 'like'), (79589, 1, 'love')]

In [37]:
# sort the words in the query based on their frequency
freqs.sort()

In [38]:
def intersect(postings1, postings2, k):
    i = 0
    j = 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] - postings2[j] == k:
            result.append(postings1[i])
            i += 1
            j += 1
        elif postings1[i] - postings2[j] < k:
            i += 1
        else:
            j += 1
    return result

In [39]:
# choose the first word in the query as the static word to compare with the other words
static = freqs[0][2]
static_postings = inverted_index[static][1]
static_postings_keys = list(static_postings.keys())

static_index = freqs[0][1]

In [40]:
FREQ_LENGTH = len(freqs)

In [43]:
# get the common postings list for the query words
def get_common_postings():
    intersection = None

    for i in range(FREQ_LENGTH - 1):

        # set the dynamic word to compare with the static word
        dynamic = freqs[i + 1][2]
        dynamic_postings = inverted_index[dynamic][1]
        dynamic_postings_keys = list(dynamic_postings.keys())

        # get the common postings list for the static and dynamic word
        if intersection is None: intersection = intersect(static_postings_keys, dynamic_postings_keys, 0)
        else:                    intersection = intersect(intersection, dynamic_postings_keys, 0)

        # if there are no common postings, break the loop
        if len(intersection) == 0: break

    return intersection

In [44]:
intersection = get_common_postings()

In [45]:
# get the ordered positions for the query words
def get_ordered_positions():
    orders = {}

    for i in range(FREQ_LENGTH - 1):

        # set the dynamic word to compare with the static word
        dynamic_index = freqs[i + 1][1]
        dynamic = freqs[i + 1][2]
        dynamic_postings = inverted_index[dynamic][1]

        # first iteration
        # proximity queries can only be iterated once
        if len(orders) == 0:
            for id in intersection:
                static_positions = static_postings[id]
                dynamic_positions = dynamic_postings[id]

                if q_type == 'PHRASE':      order = intersect(static_positions, dynamic_positions, static_index - dynamic_index)
                elif q_type == 'PROXIMITY': order = intersect(static_positions, dynamic_positions, k) + intersect(dynamic_positions, static_positions, k)

                if len(order) > 0:
                    orders[id] = order

        # subsequent iterations
        # check if the positions of the query words are in order
        else:
            for id in orders.copy():
                order = orders[id]
                dynamic_positions = dynamic_postings[id]

                # get the positions of the query words that are in order
                update = intersect(order, dynamic_positions, static_index - dynamic_index)

                if len(update) > 0: 
                    orders[id] = update
                else:
                    del orders[id]

        # if there are no wanted positions, break the loop
        if len(orders) == 0: break
    
    return orders

In [46]:
orders = get_ordered_positions()   

In [47]:
# print document ids that contain the query words in the correct order
list(orders.keys())

[484,
 2021,
 4183,
 5285,
 5493,
 7011,
 7544,
 8318,
 8413,
 11237,
 11910,
 12006,
 12241,
 13043,
 13511,
 13571,
 13951,
 14450,
 16041,
 16150,
 21626,
 23423,
 23649,
 26323,
 28143,
 30009,
 31519,
 33255,
 34989,
 35374,
 36454,
 36825,
 37723,
 39129,
 39973,
 40282,
 41455,
 41604,
 41644,
 41945,
 42219,
 42880,
 42904,
 44941,
 47293,
 49933,
 54177,
 54819,
 56629,
 57013,
 58637,
 59209,
 60336,
 60936,
 62821,
 66343,
 67046,
 68487,
 68823,
 77327,
 77649,
 79029,
 80397,
 80446,
 81028,
 81044,
 82367,
 84513,
 87118,
 87549,
 87652,
 87776,
 88401,
 89797,
 91137,
 91347,
 91421,
 94023,
 99004,
 103088,
 103950,
 104274,
 107666,
 109396,
 109884,
 110936,
 111132,
 111489,
 112589,
 112755,
 113413,
 113468,
 113658,
 113750,
 113788,
 114560,
 116072,
 117774,
 118208,
 119729,
 119849,
 120369,
 121023,
 121975,
 122544,
 123392,
 124955,
 128664,
 129234,
 129549,
 130202,
 131496,
 131936,
 132090,
 133751,
 134538,
 136147,
 138914,
 141262,
 144366,
 144772,
